[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kgweber-cwru/coding-with-ai-wn26/blob/main/week-2-conversations/concepts.ipynb)

# Week 2: Building Conversations

## Learning Objectives
By the end of this session, you will:
- Understand how conversation history works
- Build multi-turn conversations that maintain context
- Use system prompts effectively to shape behavior
- Manage conversation length and costs
- Handle different roles (system, user, assistant)

## Setup

In [ ]:
import os
import sys
from pathlib import Path

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install -q google-genai google-auth python-dotenv
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
else:
    def find_service_account_json(max_up=6):
        p = Path.cwd()
        for _ in range(max_up):
            candidate = p / "series-2-coding-llms" / "creds"
            if candidate.exists():
                for f in candidate.glob("*.json"):
                    return str(f.resolve())
            candidate2 = p / "creds"
            if candidate2.exists():
                for f in candidate2.glob("*.json"):
                    return str(f.resolve())
            p = p.parent
        return None

    sa_path = find_service_account_json()
    if sa_path:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    else:
        try:
            from dotenv import load_dotenv
            load_dotenv()
        except Exception:
            pass


In [2]:
import google.auth
from google import genai
from google.genai import types

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
client = genai.Client(vertexai=True, project=project, location="us-central1")
print(f"Using project: {project}")

print("✅ Environment loaded successfully!")

Using project: coding-with-ai-wn-26
✓ Environment loaded successfully!


## Part 1: Understanding Conversation Structure

### The Messages List
Conversations are lists of messages exchanged between the user and the model. 
- **System Instruction**: Sets the behavior/persona (passed separately in configuration)
- **User**: The human input
- **Model**: The AI response

We can track history in a simple list:
```python
messages = [
    {"role": "user", "content": "Hello!"},
    {"role": "model", "content": "Hi! How can I help?"},
    {"role": "user", "content": "Tell me about Python."}
]
```

### A Simple Two-Turn Conversation

In [3]:
# System instruction handles the persona
system_instruction = "You are a helpful teaching assistant."

# Start with first user message
messages = [
    {"role": "user", "content": "What is a variable in programming?"}
]

# Get first response
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=[types.Content(role=m["role"], parts=[types.Part(text=m["content"])]) for m in messages],
    config=types.GenerateContentConfig(system_instruction=system_instruction)
)

first_answer = response.text
print("Model:", first_answer)
print("\n" + "="*50 + "\n")

# Add model's response to history
messages.append({"role": "model", "content": first_answer})

# Add follow-up question
messages.append({"role": "user", "content": "Can you give me an example in Python?"})

# Get second response - it remembers context!
response = client.models.generate_content(
    model="gemini-2.5-flash-lite",
    contents=[types.Content(role=m["role"], parts=[types.Part(text=m["content"])]) for m in messages],
    config=types.GenerateContentConfig(system_instruction=system_instruction)
)

print("Model:", response.text)

Model: A **variable** in programming is like a **container** or a **placeholder** that holds a piece of **data**. Think of it as a labeled box where you can store information that your program can use and change.

Here's a breakdown of the key aspects of a variable:

*   **Name (Identifier):** Each variable has a unique name that you give it. This name is used to refer to the variable and access the data it holds. For example, `age`, `userName`, `totalScore`, or `isLoggedIn`.

*   **Value:** This is the actual data that the variable stores. The value can be different types of information, such as:
    *   **Numbers:** Integers (like `10`, `-5`) or floating-point numbers (like `3.14`, `0.5`).
    *   **Text (Strings):** Sequences of characters (like `"Hello, World!"`, `"Alice"`).
    *   **Booleans:** Truth values, either `true` or `false`.
    *   **More complex data structures:** Like lists, arrays, objects, etc.

*   **Data Type:** In many programming languages, variables have a spec

## Part 2: Building a Conversation Manager

Let's create a helper class to manage conversations:

In [4]:
class Conversation:
    """A simple conversation manager"""
    
    def __init__(self, system_message="You are a helpful assistant.", model="gemini-2.5-flash-lite"):
        self.system_message = system_message
        self.messages = [] # History of user/model turns
        self.model = model
        self.total_tokens = 0
    
    def add_user_message(self, content):
        """Add a user message to the conversation"""
        self.messages.append({"role": "user", "content": content})
    
    def get_response(self, temperature=0.7, max_tokens=None):
        """Get assistant response and add to history"""
        # Convert internal message format to Vertex AI Content objects
        content_list = [
            types.Content(role=m["role"], parts=[types.Part(text=m["content"])]) 
            for m in self.messages
        ]

        config = types.GenerateContentConfig(
            system_instruction=self.system_message,
            temperature=temperature,
            max_output_tokens=max_tokens
        )
        
        response = client.models.generate_content(
            model=self.model,
            contents=content_list,
            config=config
        )
        
        model_message = response.text
        self.messages.append({"role": "model", "content": model_message})
        
        if response.usage_metadata:
            self.total_tokens += response.usage_metadata.total_token_count
        
        return model_message
    
    def chat(self, user_message, temperature=0.7, max_tokens=None):
        """Convenience method: add user message and get response"""
        self.add_user_message(user_message)
        return self.get_response(temperature, max_tokens)
    
    def display_history(self):
        """Display the conversation history"""
        print(f"SYSTEM: {self.system_message}")
        print("-" * 50)
        for msg in self.messages:
            role = msg["role"].upper()
            content = msg["content"]
            print(f"{role}: {content}")
            print("-" * 50)
    
    def get_token_count(self):
        """Get total tokens used"""
        return self.total_tokens

print("✅ Conversation class created!")

✓ Conversation class created!


### Test the Conversation Manager

In [5]:
# Create a conversation with a specific persona
convo = Conversation(
    system_message="You are a friendly data science tutor. Keep answers concise but clear."
)

# Have a multi-turn conversation
print(convo.chat("What's the difference between supervised and unsupervised learning?"))
print("\n" + "="*50 + "\n")

print(convo.chat("Which one would I use for clustering?"))
print("\n" + "="*50 + "\n")

print(convo.chat("Give me an example algorithm for that."))
print("\n" + "="*50 + "\n")

print(f"Total tokens used: {convo.get_token_count()}")

Think of it like this:

*   **Supervised Learning:** You're given a dataset with **labeled examples**. It's like having a teacher showing you pictures of cats and dogs, and telling you which is which. The goal is to learn a mapping from input to output.

*   **Unsupervised Learning:** You're given a dataset with **no labels**. It's like being shown a pile of photos and asked to group them by similarity without being told what the groups should be. The goal is to find hidden patterns or structures in the data.


You would use **unsupervised learning** for clustering.

Clustering is all about finding groups or clusters within unlabeled data, which is the core idea of unsupervised learning.


A classic example of an unsupervised learning algorithm for clustering is **K-Means Clustering**.


Total tokens used: 535


### View Full Conversation History

In [6]:
convo.display_history()

SYSTEM: You are a friendly data science tutor. Keep answers concise but clear.
--------------------------------------------------
USER: What's the difference between supervised and unsupervised learning?
--------------------------------------------------
MODEL: Think of it like this:

*   **Supervised Learning:** You're given a dataset with **labeled examples**. It's like having a teacher showing you pictures of cats and dogs, and telling you which is which. The goal is to learn a mapping from input to output.

*   **Unsupervised Learning:** You're given a dataset with **no labels**. It's like being shown a pile of photos and asked to group them by similarity without being told what the groups should be. The goal is to find hidden patterns or structures in the data.
--------------------------------------------------
USER: Which one would I use for clustering?
--------------------------------------------------
MODEL: You would use **unsupervised learning** for clustering.

Clustering is

## Part 3: System Instruction Strategies

The system instruction is powerful! Let's explore different personas:

In [7]:
# Persona 1: Concise expert
expert = Conversation(
    system_message="You are an expert who gives concise, technical answers. Use precise terminology."
)

# Persona 2: Beginner-friendly teacher
teacher = Conversation(
    system_message="You are a patient teacher explaining concepts to complete beginners. Use analogies and simple language."
)

# Same question to both
question = "What is a neural network?"

print("EXPERT:")
print(expert.chat(question))
print("\n" + "="*50 + "\n")

print("TEACHER:")
print(teacher.chat(question))

EXPERT:
A neural network is a computational model inspired by the structure and function of biological neural networks. It consists of interconnected processing units called neurons, organized in layers. These neurons receive input signals, apply a weighted sum and an activation function to these inputs, and produce an output signal. The connections between neurons have associated weights, which are adjusted during a training process to learn patterns and relationships within data.


TEACHER:
Imagine you're trying to teach a little kid to recognize a cat. How would you do it?

You'd show them pictures of cats, right? And you'd say, "This is a cat." You'd show them different cats – fluffy ones, sleek ones, cats with pointy ears, cats with folded ears. You'd also show them things that are *not* cats, like dogs, birds, or chairs, and say, "This is not a cat."

Over time, the kid starts to pick up on what makes a cat a cat. They learn about furry textures, whiskers, pointy ears, and a cert

### Structured Output with System Instructions

In [8]:
# Request specific output format
structured = Conversation(
    system_message="""You are a medical information assistant. 
    Always structure your responses as:
    1. DEFINITION: Brief definition
    2. KEY POINTS: 3-4 bullet points
    3. NOTE: Important consideration or caution
    """
)

print(structured.chat("What is hypertension?"))
print("\n" + "="*50 + "\n")
print(structured.chat("What about hypotension?"))

1.  **DEFINITION:** Hypertension, commonly known as high blood pressure, is a chronic medical condition in which the blood pressure in the arteries is persistently elevated.

2.  **KEY POINTS:**
    *   Blood pressure is measured in millimeters of mercury (mmHg) and has two numbers: systolic pressure (the top number, representing pressure when the heart beats) and diastolic pressure (the bottom number, representing pressure when the heart rests between beats).
    *   Hypertension is generally diagnosed when blood pressure readings are consistently at or above 130/80 mmHg.
    *   It often has no symptoms and is sometimes called the "silent killer" because damage can occur without people knowing.
    *   Uncontrolled hypertension significantly increases the risk of serious health problems like heart disease, stroke, kidney failure, and vision loss.

3.  **NOTE:** While lifestyle factors (diet, exercise, weight, stress, alcohol intake) play a significant role, hypertension can also be i

## Part 4: Managing Context Window

Conversations can get too long! The model has a maximum context window (tokens it can process).

### Strategy 1: Keep Recent Messages Only

In [9]:
class ConversationWithLimit(Conversation):
    """Conversation that keeps only recent messages"""
    
    def __init__(self, system_message="You are a helpful assistant.", 
                 model="gemini-2.5-flash-lite", max_history=6):
        super().__init__(system_message, model)
        self.max_history = max_history  # Keep last N messages
    
    def get_response(self, temperature=0.7, max_tokens=None):
        # Keep only last N messages
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]
        
        return super().get_response(temperature, max_tokens)

# Test it
limited = ConversationWithLimit(max_history=4)

for i in range(6):
    response = limited.chat(f"This is message number {i+1}")
    print(f"Turn {i+1}: {response[:50]}...")

print("\n" + "="*50 + "\n")
print(f"Messages in memory: {len(limited.messages)}")
print("\nCurrent history:")
limited.display_history()

Turn 1: Okay, I understand. This is message number 1. What...
Turn 2: Got it. Message number 2. What's next?...
Turn 3: Acknowledged. Message number 3. How can I help you...
Turn 4: Understood. Message number 4 received. What would ...
Turn 5: Acknowledged. Message number 5. What else can I do...
Turn 6: Understood. Message number 6. How can I assist you...


Messages in memory: 5

Current history:
SYSTEM: You are a helpful assistant.
--------------------------------------------------
MODEL: Understood. Message number 4 received. What would you like to do now?
--------------------------------------------------
USER: This is message number 5
--------------------------------------------------
MODEL: Acknowledged. Message number 5. What else can I do for you?
--------------------------------------------------
USER: This is message number 6
--------------------------------------------------
MODEL: Understood. Message number 6. How can I assist you further?
---------------------------------

### Strategy 2: Summarize Old Context

In [10]:
def summarize_conversation(messages):
    """Create a summary of the conversation so far"""
    # Format the conversation text
    convo_text = "\n".join([
        f"{msg['role']}: {msg['content']}" 
        for msg in messages
    ])
    
    summary_prompt = f"""Summarize this conversation in 2-3 sentences, 
    preserving key facts and context:
    
    {convo_text}
    """
    
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=[types.Content(role="user", parts=[types.Part(text=summary_prompt)])],
        config=types.GenerateContentConfig(
            system_instruction="You create concise conversation summaries.",
            temperature=0.3
        )
    )
    
    return response.text

# Test it
print(summarize_conversation(limited.messages))

The user is sequentially sending messages, numbering them 4, 5, and 6. The model acknowledges each message and prompts the user for further instructions.


## Part 5: Practical Conversation Applications

### Application 1: Q&A Assistant

In [11]:
class QAAssistant:
    """Interactive Q&A assistant with memory"""
    
    def __init__(self, topic="general knowledge"):
        system_msg = f"""You are a knowledgeable assistant specialized in {topic}. 
        Answer questions clearly and build on previous context in the conversation.
        If you don't know something, say so."""
        self.convo = Conversation(system_message=system_msg)
        self.topic = topic
    
    def ask(self, question):
        return self.convo.chat(question)
    
    def history(self):
        self.convo.display_history()

# Create a Python programming assistant
python_helper = QAAssistant(topic="Python programming")

print(python_helper.ask("What are list comprehensions?"))
print("\n" + "="*50 + "\n")

print(python_helper.ask("Show me an example with filtering."))
print("\n" + "="*50 + "\n")

print(python_helper.ask("How is that different from a regular for loop?"))

List comprehensions are a concise and elegant way to create lists in Python. They offer a shorter syntax for creating a new list based on the values of an existing iterable (like a list, tuple, or string).

Think of them as a more readable and efficient alternative to traditional `for` loops when you want to generate a list.

Here's the basic syntax:

```python
new_list = [expression for item in iterable if condition]
```

Let's break down the components:

*   **`expression`**: This is what gets evaluated for each `item` in the `iterable`. It's the value that will be added to the new list.
*   **`for item in iterable`**: This is the core loop. It iterates through each element (`item`) in the `iterable`.
*   **`if condition` (optional)**: This is a filter. If the `condition` evaluates to `True` for a given `item`, then the `expression` is evaluated and added to the new list. If the `condition` is omitted, every `item` from the `iterable` will be processed.

**Why use list comprehensions

### Application 2: Research Interview Assistant

In [12]:
class InterviewAssistant:
    """Helps conduct and document research interviews"""
    
    def __init__(self, research_topic):
        system_msg = f"""You are helping conduct a research interview about {research_topic}.
        Your role is to:
        1. Ask thoughtful follow-up questions
        2. Clarify ambiguous statements
        3. Probe for more details when needed
        4. Maintain a professional, curious tone
        """
        self.convo = Conversation(system_message=system_msg)
        self.topic = research_topic
    
    def respond(self, interviewee_response):
        """Process interviewee response and ask follow-up"""
        return self.convo.chat(interviewee_response)
    
    def get_summary(self):
        """Get a summary of key points from the interview"""
        return summarize_conversation(self.convo.messages)

# Example usage
interviewer = InterviewAssistant("patient experiences with telemedicine")

print("INTERVIEWER:", interviewer.respond("I started using telemedicine during COVID."))
print("\n" + "="*50 + "\n")

print("INTERVIEWER:", interviewer.respond("It was convenient but I missed the personal connection."))
print("\n" + "="*50 + "\n")

print("Interview Summary:")
print(interviewer.get_summary())

INTERVIEWER: Thank you for sharing that. Could you tell me a bit more about what prompted you to try telemedicine at that time? Was it a specific need, or more of a general shift in how you accessed healthcare?


INTERVIEWER: That's a very common sentiment. Can you elaborate on what you mean by "personal connection"? What aspects of an in-person visit do you feel are harder to replicate through a screen?


Interview Summary:
The user began using telemedicine during COVID for convenience but missed the personal connection of in-person visits. They are seeking to understand what specific aspects of that personal connection are difficult to replicate virtually.


### Application 3: Debugging Assistant

In [13]:
debugging_assistant = Conversation(
    system_message="""You are a debugging assistant. When users share code and errors:
    1. Identify the likely cause
    2. Explain why it's happening
    3. Suggest a fix with code
    4. Ask clarifying questions if needed
    """
)

# Simulate debugging session
error_report = """I'm getting a KeyError in my Python code:
my_dict = {'name': 'Alice', 'age': 30}
print(my_dict['city'])
"""

print(debugging_assistant.chat(error_report))
print("\n" + "="*50 + "\n")

print(debugging_assistant.chat("How can I check if a key exists before accessing it?"))

This `KeyError: 'city'` is happening because you're trying to access a key that doesn't exist in your dictionary.

**Explanation:**

Dictionaries in Python store data as key-value pairs. When you try to retrieve a value using a key (like `my_dict['city']`), Python looks for that specific key within the dictionary. If the key isn't found, it raises a `KeyError` to let you know that you're asking for something that isn't there.

In your case, `my_dict` only contains the keys `'name'` and `'age'`. There is no key named `'city'`.

**Suggested Fix:**

To avoid this error, you should either:

1.  **Add the key to the dictionary before trying to access it:**

    ```python
    my_dict = {'name': 'Alice', 'age': 30}
    my_dict['city'] = 'New York' # Add the 'city' key and its value
    print(my_dict['city'])
    ```

2.  **Check if the key exists before accessing it:**

    ```python
    my_dict = {'name': 'Alice', 'age': 30}
    if 'city' in my_dict:
        print(my_dict['city'])
    else:


## Part 6: Cost and Performance Considerations

In [14]:
# Compare conversation lengths
short_convo = Conversation()
for i in range(3):
    short_convo.chat(f"Question {i+1}")

long_convo = Conversation()
for i in range(10):
    long_convo.chat(f"Question {i+1}")

print(f"Short conversation (3 turns): {short_convo.get_token_count()} tokens")
print(f"Long conversation (10 turns): {long_convo.get_token_count()} tokens")
print(f"\nToken growth factor: {long_convo.get_token_count() / short_convo.get_token_count():.2f}x")

Short conversation (3 turns): 169 tokens
Long conversation (10 turns): 1861 tokens

Token growth factor: 11.01x


## Part 7: New API - built-in chat

The latest versions of the Google SDK have a built in chat object that stores history for you. It's still worthwhile to work through this with the hand-built class to be sure you understand what's going on, though. Here's an example of how to use the Google API

This is just a wrapper around the `generateContent` tool - it's still sending the entire context back and forth with each step of the conversation.

In [15]:
chat = client.chats.create(model='gemini-2.5-flash')

# --- Turn 1 ---
chat.send_message("Hi, I'm building a project using the Gemini Python SDK.")

# --- Turn 2 ---
chat.send_message("What is the difference between a 'Content' object and a 'Part'?")

# --- Turn 3 ---
response = chat.send_message("Can you give me a code example of a multi-part message?")

print(f"Latest Response: {response.text}")

Latest Response: Okay, let's create a clear code example of a multi-part message using the Gemini Python SDK.

This example will demonstrate sending a text prompt along with *two* images in a single `Content` object to the `gemini-pro-vision` model.

**Prerequisites:**

1.  **Google Generative AI SDK:**
    ```bash
    pip install google-generativeai
    ```
2.  **Pillow (for image handling):**
    ```bash
    pip install Pillow
    ```
3.  **Requests (to download example images):**
    ```bash
    pip install requests
    ```
4.  **An API Key:** Make sure you have your Gemini API key from Google AI Studio.

---

```python
import google.generativeai as genai
import PIL.Image
import requests
import io
import os

# --- 1. Configure your API Key ---
# Replace with your actual API key
# It's best practice to load this from an environment variable
# genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
genai.configure(api_key="YOUR_API_KEY") # <-- Replace this!

# --- 2. Initialize the 

Let's Look at the history:

In [20]:
for message in chat.get_history():
    # Each message is a 'Content' object
    print(f"Role: {message.role}")
    for part in message.parts:
        # Each part could be text, inline_data (blobs), or function_calls
        if part.text:
            print(f"Text: {part.text[:50]}...")
    print("-" * 20)

Role: user
Text: Hi, I'm building a project using the Gemini Python...
--------------------
Role: model
Text: Great! I'm ready to help you with your project usi...
--------------------
Role: user
Text: What is the difference between a 'Content' object ...
--------------------
Role: model
Text: Okay, let's clarify the difference between `Conten...
--------------------
Role: user
Text: Can you give me a code example of a multi-part mes...
--------------------
Role: model
Text: ```python
import google.generativeai as genai
impo...
--------------------


You could manipulate that history and send a trimmed or modified version to a new conversation:

In [17]:
previous_session = [
    types.Content(role="user", parts=[types.Part(text="My favorite color is Blue.")]),
    types.Content(role="model", parts=[types.Part(text="Understood! I will remember that.")])
]

# Start a new chat with that memory pre-loaded
new_chat = client.chats.create(model='gemini-2.0-flash', history=previous_session)

In [19]:
response = new_chat.send_message("What is my favorite color?")
print(f"Response: {response.text}")

Response: Blue



## Key Takeaways

1. **Conversations are message lists** - Just add to the list to maintain context
2. **System messages are powerful** - They shape the entire conversation behavior
3. **Context grows quickly** - Each turn includes all previous messages
4. **Manage conversation length** - Keep recent messages or summarize old ones
5. **Structure matters** - Clear roles and formatting help the model respond appropriately

## Next Week Preview

Next week, we'll explore **programmatic prompt engineering**:
- Building dynamic prompts
- Template systems
- Few-shot learning
- Output parsing

Complete the assignment to practice building conversational applications!